# Lernaufgabe 2: Hamming-Kodierung

Sie kennen bereits die Hamming-Kodierung aus dem vorherigen Kapitel. Zur Vertiefung des Gelernten, und um`jupyter` kennenzulernen, implementieren Sie sie in dieser und den folgenden Lernaufgaben mit `python`.

🤔 Doch wie funktioniert `jupyter`? Jede Datei in `jupyter` ist ein Notizbuch, *notebook* zu Englisch, welches aus *Zellen* aufgebaut ist. Jede Zelle kann entweder Text oder `python`-Code enthalten. Wenn Sie eine Code-Zelle anwählen, und die Tasten ⬆️ + ⏎ drücken, wird der Code in dieser Zelle ausgeführt, und das Resultat darunter angezeigt. Das Bemerkenswerte hierbei ist, dass die Berechnungen und deren Resultate aus einer Zelle in einer anderen Zelle weiterverarbeitet werden können.

👏 Ihre Aufgabe ist nun jeweils, die Kommentare im Format `# TODO: ...` durch funktionierenden Code zu ersetzen.

## Teilaufgabe 2.1

Damit wir testen können, ob Ihre Implementierung am Ende auch tatsächlich funktioniert, müssen wir ein *Testgeschirr* haben; in unserem Fall die Simulation eines unsicheren Mediums sowie verschiedene Funktionen zur Kodierung und Dekodierung der Nachrichten. Nachfolgend verwenden wir folgende Terminologie:

* Die Nachricht `message` ist ein Text, der für Menschen lesbar ist
* Diese hat eine Repräsentation als `bits`, also eine Folge von `0` und `1`. In der nachfolgenden Implementation verwenden wir hierzu die Datenstruktur eines `bitarray`s, welche sich sehr ähnlich wie das Ihnen bereits bekannte `array` verhält.
* In Anlehnung an die Erklärung mit dem Kartentrick aus dem Buch können wir diese `bits` auch als `table` darstellen, welches in unserem Fall ein `array` von `bitarray`s ist.

⚠️ Der Fokus in diesem Kapitel liegt auf einem vertieften Verständnis der selbstkorrigierenden Kodierungen. Wir lassen darum das Problem der Suche nach einer optimalen Dartstellungsform der Nachricht außen vor, und nehmen für unsere Kartentrick-Tabelle immer eine fixe Anzahl von Spalten, nämlich `4`.

In groben Zügen läuft der Test, ob die Hamming-Kodierung richtig implementiert wurde, wie folgt ab:

1. Die natürlichsprachliche Nachricht `message` wird zu `bits` und zum `table` konvertiert
2. Daraufhin wird sie kodiert (`encoded_...`)
3. Für die Übertragung wird sie durch ein unzuverlässiges Medium geschickt, in welchem einige Bits verändert werden können (`corrupted_...`)
4. Danach wird versucht, den Fehler zu korrigieren (`corrected_...`)
5. Die `corrected_bits` und `corrected_table` werden zur `decoded_message` konvertiert

In [6]:
from bitarray import bitarray

def message_to_bits(message):
    """
    Konvertiert einen gegebenen string in seine Repräsentation als Reihe 
    von bits in Form eines bitarray.
    """
    bits = bitarray()
    bits.frombytes(message.encode("ascii"))
    return bits

def bits_to_message(bits):
    """
    Versucht eine Reihe von bits in Form eines bitarray in einen string zu konvertieren.
    bit-Folgen, die keinem Symbol zugeordnet werden können, werden in ein � umgewandelt.
    """
    bees = bits.tobytes()
    message = ""
    for bee in bees:
        try:
            message += bytes([bee]).decode("ascii")
        except:
            message += "�"
    return message

assert message_to_bits("Hello") == bitarray('0100100001100101011011000110110001101111')
assert bits_to_message(bitarray('0100100001100101011011000110110001101111')) == "Hello"

In der obigen Zelle erhalten Sie zwei Helfer-Funktionen, welche Ihnen den Umgang mit bits erleichtern. Wir verwenden zur Darstellung von bit-Folgen die Bibliothek [`bitarray`](https://pypi.org/project/bitarray/), welche uns dem Umgang damit erleichtert. Den genauen Umgang damit müssen Sie nicht verstehen, es reicht aus zu wissen, dass Sie das $i$-te Element eines `bitarray` mit `bitarray[i]` - ganz so, wie Sie es sich von `array`s gewohnt sind, ansprechen können.

Wenn Sie die Zelle ausführen, indem Sie die Tasten ⬆️ + ⏎ drücken, dann stehen Ihnen die Funktionen auch in nachfolgenden Code-Zellen zur Verfügung. 

☝️ Die `assert`-Befehle am Ende der Zelle können Sie als Miniatur-Testfälle begreifen. Diese beweisen, dass die Funktionen tatsächlich das tun, was Sie zu tun vorgeben. Der Befehl `assert x` gibt einen Fehler aus, falls `x`, welches hier für eine beliebige Überprüfung steht, `false` ist, und verhält sich sonst still.

In [1]:
# TODO: Benutzen Sie die Funktionen, um die bit-Folge für "Hallo Welt" herauszufinden,
# indem Sie in diese Zelle hier den entsprechenden Code schreiben und mit ⬆️ + ⏎ ausführen.

# Hier sollte Ihr Code stehen!

## Teilaufgabe 2.2

Nachfolgend erhalten Sie ein paar weitere Helfer-Funktionen, welche Sie nachfolgend benötigen werden. Hierbei legen wir bei unseren Nachrichten $m = m_0, m_1, m_2, ...$ fest, dass Sie wie folgt in Tabellen-Form übertragen werden (und vice-versa!).

⚠️ Erinnern Sie sich, dass für gewöhnlich in der Mathematik das erste Element einer Nachricht mit $1$ bezeichnet wird, in `python` aber mit `0`! Wir verwenden der Einheitlichkeit wegen nachfolgend letztere Nummerierung in beiden Notationen.

0️⃣ Falls die Nachricht nicht genau eine Länge $|m|\mod{4} \ 0$ hat, so werden die verbleibenden bits mit `0` aufgefüllt. Wird die Tabelle zurück in ein `bitarray` konvertiert, können diese `0` bedenkenlos beibehalten werden.

| | | | |
| --- | --- | --- | --- |
| $m_0$ | $m_1$ | $m_2$ | $m_3$ |
| $m_4$ | $m_5$ | $m_6$ | $m_7$ |
| $m_8$ | $m_9$ | $m_{10}$ | $m_{11}$ |
| $m_{12}$ | $0$ | $0$ | $0$ |

In [7]:
import math

def empty_table(rows, columns):
    """
    Erstellt eine leere Tabelle als Array von bitarray in den
    Dimensionen rows ⨯ columns, und befüllt alle bits mit 0
    """
    table = []
    for i in range(0, rows):
        row = bitarray(columns)
        row.setall(0)
        table.append(row)
    return table

def copy_table(tible):
    """
    Erstellt eine Kopie von einer gegebenen Tabelle. Dies ist 
    beispielsweise nötig, wenn wir auf einer Tabelle Operationen
    vornehmen möchten, aber uns ein unverändertes Original be-
    halten möchten.
    """
    table = []
    for row in tible:
        table.append(bitarray([ bit for bit in row ]))
    return table

def print_table(table):
    """
    Ermöglicht uns, Tabellen einfach für einen Menschen
    zu lesen darzustellen
    """
    for row in table:
        for bit in row:
            print(bit, end=" ")
        print()
    print()
        
def get_column(table, index):
    """
    Liefert alle bits in einer Zeile einer Tabelle als bitarray zurück
    """
    column = bitarray()
    for row in table:
        column.append(row[index])
    return column

def bits_to_table(bits, columns=4):
    """
    Wandelt ein bitarray in eine Tabelle mit columns Spalten.
    Falls kein columns-Wert angegeben wird, dann wird 4 als Standart-
    Wert angenommen.
    """
    rows = math.ceil(len(bits) / columns)
    table = empty_table(rows, columns)
        
    for i, bit in enumerate(bits):
        row = int(i / columns)
        table[row][i % columns] = bit
    return table

def table_to_bits(table):
    """
    Wandelt eine Tabelle in ein einzelnes bitarray um,
    in dem es ein neues bitarray erstellt, und für jede Zeile
    jedes bit anhängt.
    """
    
    # TODO: Vervollständigen Sie die nachfolgende Funktion.
    # Verwenden Sie hierzu bits.append(). Die nachfolgenden
    # assert-Befehle sollten keine Fehler mehr ausgeben.
    bits = bitarray()
    
    # Hier sollte Ihr Code stehen!
    
    return bits

table = empty_table(5, 5)
assert len(table) == 5
assert len(table[2]) == 5
bits = bitarray('0100100001100101011011000110110001101111')
table = bits_to_table(bits)
assert len(table) == 10
assert bits == table_to_bits(bits_to_table(bits))
assert get_column(table,0) == bitarray('0100010101')

AssertionError: 